In [1]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import CNN6AutoEncoder as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

In [2]:
with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

#log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
#logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# Setting seed
modeler.set_seed(42)

In [4]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
"""
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
"""

'\nINPUT_ROOT = config[\'IO_OPTION\'][\'INPUT_ROOT\']\ndev_path = INPUT_ROOT + "/dev_data"\nadd_dev_path = INPUT_ROOT + "/add_dev_data"\n# machine type\nMACHINE_TYPE = config[\'IO_OPTION\'][\'MACHINE_TYPE\']\nmachine_types = os.listdir(dev_path)\n# output dirs\nOUTPUT_ROOT = config[\'IO_OPTION\'][\'OUTPUT_ROOT\']\nMODEL_DIR = config[\'IO_OPTION\'][\'OUTPUT_ROOT\'] + \'/models\'\nTB_DIR = config[\'IO_OPTION\'][\'OUTPUT_ROOT\'] + \'/tb\'\nPKL_DIR = OUTPUT_ROOT + \'/pkl\'\nos.makedirs(MODEL_DIR, exist_ok=True)\nos.makedirs(TB_DIR, exist_ok=True)\n'

In [5]:
path = '/media/hiroki/working/research/dcase2020/datasets/DCASE2/dev_data/fan/train/normal_id_00_00000000.wav'
sound_data = com.file_load(path,
                           sr=config['preprocessing']['sample_rate'],
                           mono=config['preprocessing']['mono'])

In [6]:
net = Model(sample_rate=config['preprocessing']['sample_rate'],
            window_size=config['preprocessing']['window_size'],
            hop_size=config['preprocessing']['hop_size'],
            mel_bins=config['preprocessing']['mel_bins'],
            fmin=config['preprocessing']['fmin'],
            fmax=config['preprocessing']['fmax'])

In [7]:
from torchsummary import summary
net = net.to('cuda:0')
summary(net, (sound_data[1],)) # summary(model,(channels,H,W))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 513, 129]         525,312
            Conv1d-2             [-1, 513, 129]         525,312
       Spectrogram-3          [-1, 1, 129, 513]               0
  LogmelFilterBank-4           [-1, 1, 129, 64]               0
       BatchNorm2d-5           [-1, 64, 128, 1]             128
            Conv2d-6          [-1, 64, 128, 64]           1,600
       BatchNorm2d-7          [-1, 64, 128, 64]             128
      ConvBlock5x5-8           [-1, 64, 64, 32]               0
            Conv2d-9          [-1, 128, 64, 32]         204,800
      BatchNorm2d-10          [-1, 128, 64, 32]             256
     ConvBlock5x5-11          [-1, 128, 32, 16]               0
           Conv2d-12          [-1, 256, 32, 16]         819,200
      BatchNorm2d-13          [-1, 256, 32, 16]             512
     ConvBlock5x5-14           [-1, 256